In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import torch
from tqdm.notebook import tqdm
from transformers import BertTokenizer
from torch.utils.data import TensorDataset
from transformers import BertForSequenceClassification
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import f1_score
import random
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from imblearn.over_sampling import SMOTE
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [7]:
file = open("merged_data_with_got_ocr.csv", 'r')
df = pd.read_csv(file)

df['Institutional_Form'] = df['Institutional_Form'].str.lower()
df['Institutional_Form'] = df['Institutional_Form'].replace('cooperative purchasing agreement', 'cooperative purchasing arrangements')
df['Institutional_Form'] = df['Institutional_Form'].replace('transfer of authority', 'transfer of functions')
df['Institutional_Form'] = df['Institutional_Form'].replace('transfer of responsibility', 'transfer of functions')
df['Institutional_Form'] = df['Institutional_Form'].replace('joint facilities', 'joint facilities/co-location')
df['Institutional_Form'] = df['Institutional_Form'].replace('joint production', 'Joint Operations')
df['Institutional_Form'] = df['Institutional_Form'].replace('joint facilities/co-location and joint operations', 'joint facilities/co-location')
df['Institutional_Form'] = df['Institutional_Form'].replace('cooperative purchasing agreement and new joint entities', 'cooperative purchasing arrangements')
df['Institutional_Form'] = df['Institutional_Form'].replace('service contract and grants/other subsidies', 'service contract')
df['Institutional_Form'].value_counts()

Institutional_Form
service contract                       555
joint operations                       172
new joint entities                     120
resource sharing                        76
joint facilities/co-location            29
transfer of functions                   28
grants/other subsidies                  26
cooperative purchasing arrangements     15
Joint Operations                         5
coproduction                             5
other                                    2
Name: count, dtype: int64

In [9]:
def clean_text(text):
    # Remove newline-separated letters
    cleaned_text = re.sub(r'(\n[a-zA-Z])', '', text)
    # Remove single newlines or extraneous spaces
    cleaned_text = re.sub(r'\n+', ' ', cleaned_text).strip()
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
    cleaned_text = re.sub(r'11.', ' ', cleaned_text).strip()
    cleaned_text = re.sub(r"\n\'", ' ', cleaned_text).strip()
    cleaned_text = re.sub(r'\n', ' ', cleaned_text).strip()
    ##Patterns found
    cleaned_text = re.sub(r'\n', ' ', cleaned_text).strip()
    cleaned_text = re.sub(r'\.{2,}\s{1,}', ' ', cleaned_text).strip()
    cleaned_text = re.sub(r'\.{2,}', ' ', cleaned_text).strip()
    cleaned_text = re.sub(r'\({2,}' , ' ', cleaned_text).strip()
    cleaned_text = re.sub(r'\){2,}'  , ' ', cleaned_text).strip()
    cleaned_text = re.sub(r'~{2,}', ' ', cleaned_text).strip()
    
    return cleaned_text

In [11]:
#dropping the class with only two instances
class_counts = df['Institutional_Form'].value_counts()
classes_to_drop = class_counts[class_counts <= 2].index
df = df[~df['Institutional_Form'].isin(classes_to_drop)]
df = df.reset_index(drop=True)


In [17]:
for i in range(len(df)):
    df.loc[i, 'got_ocr_text'] = clean_text(df['got_ocr_text'][i])

In [73]:
# Encode the string labels as integers
df['Institutional_Form'] = df['Institutional_Form'].str.lower()
possible_labels = df.Institutional_Form.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

df['label'] = df.Institutional_Form.replace(label_dict)

labels = df['label'].to_numpy()
text=df['got_ocr_text']

/var/folders/2l/yz2br5b90k3967p2xl16txm80000gn/T/ipykernel_4925/3042518169.py:10: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['label'] = df.Institutional_Form.replace(label_dict)


In [75]:
#BERT
X_train, X_val, y_train, y_val = train_test_split(text, 
                                                  labels, 
                                                  test_size=0.2, 
                                                  random_state=42, 
                                                  stratify=labels)

#df['data_type'] = ['not_set']*df.shape[0]

#df.loc[X_train, 'data_type'] = 'train'
#df.loc[X_val, 'data_type'] = 'val'

#df.groupby(['Institutional_Form', 'label', 'data_type']).count()

In [77]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)
                                          
encoded_data_train = tokenizer.batch_encode_plus(
    X_train, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    X_val, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(y_train)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(y_val)

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/opt/anaconda3/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:2673: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [79]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(np.unique(labels)),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [89]:
batch_size = 3

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)
                  
epochs = 10

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def f1_score_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    #le.inverse_transform
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    
    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat == label]
        y_true = labels_flat[labels_flat == label]
        
        # Calculate F1 score for the current class
        f1 = f1_score(y_true, y_preds, average='macro')
        
        print(f'Class: {label_dict_inverse[label]}')
        print(f'F1 Score: {f1:.2f}\n')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



/opt/anaconda3/lib/python3.12/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

def evaluate(dataloader_val):
    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total / len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals
    
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0
    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    
    for batch in progress_bar:
        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
    torch.save(model.state_dict(), f'finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    loss_train_avg = loss_train_total / len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')
    
    # Print F1 score for each class
    print("\nF1 Score for each class:")
    f1_score_per_class(predictions, true_vals)